# Hierarchical Indexing

Pandas has the ability to store higher dimensional data using Panel and Panel4D, but it is more common to use hierarchical indexing (multi-indexing) to compactly represent higher dimensional data within familiar Series and DF objects.

In [1]:
import pandas as pd
import numpy as np

# A Multiply Indexed Series

Consider representing two-dim data within a one-dim Series. We consider data where each point has a character and numerical key

## The bad way

In [20]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index).sort_index()
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [21]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

The convenience ends there: to select all data from 2010 we might need to do some messy and maybe slow munging:

In [22]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

## The better way: Pandas MultiIndex

In [25]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [30]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

We can now just use Pandas slicing notation:

In [34]:
pop.loc[:'California',]

California  2000    33871648
            2010    37253956
dtype: int64

In [37]:
pop.loc[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

## MultiIndex as extra dimension

We could have easily sotred the same data with a simple DataFrame, with index and column labels - it is built with this equivalence in mind. The unstack() method converts a mutiply indexed Series into a conventionally indexed DataFrame:

In [38]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [39]:
# to reverse this
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

The question of why to bother arises. Just as we can use multi-indexing to represent two-dim data within a one-dim Series, we can use it to represent three or more in a Series or DataFrame. Each extra level represents another dimension; this gives us flexibility in the data we want to represent. Suppose we want to add an extra column of demographic data e.g. population under 18. With a MultiIndex this is the same as adding another column to the dataframe:

In [42]:
pop_df = pd.DataFrame({'total': pop,
                      'under18':  [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014] })
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

Also all the ufuncs work, e.g. to compute fraction of under 18:

In [45]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## Methods of MultiIndex Creation

Simplest way is to pass a list of two or more index arrays:

In [53]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'],[1,2,1,2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.130175  0.970409
  2  0.496492  0.700120
b 1  0.566604  0.425219
  2  0.434755  0.914970

Similarly:

In [54]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

## Explicit MultiIndex constructors

In [55]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [56]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [57]:
pd.MultiIndex.from_product([['a', 'b'], [1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [59]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

## MultiIndex level names

In [60]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [61]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## MultiIndex for columns

Due to the symmetry of rows and columns we can do the same as above for cols

In [68]:
index = pd.MultiIndex.from_product([[2013, 2014], [1,2]],
                                  names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                    names=['Subject', 'Type'])

#mock data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

Subject      Bob       Guido         Sue      
Type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.0  37.3  39.0  35.9  28.0  37.2
     2      31.0  37.2  37.0  38.7  43.0  37.1
2014 1      14.0  36.2  39.0  37.2  32.0  36.1
     2      26.0  37.4  22.0  35.7  34.0  36.6

Note this is essentially four-dimensional data. We can also access all info of person:

In [70]:
health_data['Guido']

Type          HR  Temp
year visit            
2013 1      39.0  35.9
     2      37.0  38.7
2014 1      39.0  37.2
     2      22.0  35.7

## Indexing and Slicing a MultiIndex

Helps to think about the indices as added dimensions.

### Multiply Indexed Series

In [71]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [72]:
pop['California', 2000]

33871648

In [73]:
# partial indexing 

pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [74]:
# partial slicing

pop['California': 'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:

In [77]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

Boolean masks and fancy indexing both work as well:

In [78]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [79]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply Indexed DataFrames

In [80]:
health_data

Subject      Bob       Guido         Sue      
Type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.0  37.3  39.0  35.9  28.0  37.2
     2      31.0  37.2  37.0  38.7  43.0  37.1
2014 1      14.0  36.2  39.0  37.2  32.0  36.1
     2      26.0  37.4  22.0  35.7  34.0  36.6

The section above applies to the columns, since cols are primary in DataFrames:

In [81]:
health_data['Guido', 'HR']

year  visit
2013  1        39.0
      2        37.0
2014  1        39.0
      2        22.0
Name: (Guido, HR), dtype: float64

We can also use loc and iloc:

In [82]:
health_data.iloc[:2, :2]

Subject      Bob      
Type          HR  Temp
year visit            
2013 1      37.0  37.3
     2      31.0  37.2

These perform an array like view of the underlying two-dimensional data, but each index in loc or iloc can be passed a tuple of multiple indices:

In [95]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        37.0
      2        31.0
2014  1        14.0
      2        26.0
Name: (Bob, HR), dtype: float64

Trying to create a slice within a tuple will riase an error, so best way it to use an IndexSlice object

In [96]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-96-fb34fa30ac09>, line 1)

In [97]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:,'HR']]

,Subject,Bob,Guido,Sue
,Type,HR,HR,HR
year,visit,,,
2013,1,37.0,39.0,28.0
2014,1,14.0,39.0,32.0


## Rearranging Multi-Indices

### Sorted and unsorted indices

*Many of the MultiIndex slicing operations will fail if the index is not sorted*

In [98]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.825006
      2      0.326325
c     1      0.321312
      2      0.442736
b     1      0.797434
      2      0.425433
dtype: float64

In [101]:
try:
    data['a': 'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Partial slices and other similar operations require the levels to be sorted (lexographically.) Some examples to do this raee sort_index() and sortlevel().

In [103]:
data = data.sort_index()
data

char  int
a     1      0.825006
      2      0.326325
b     1      0.797434
      2      0.425433
c     1      0.321312
      2      0.442736
dtype: float64

In [104]:
data['a':'b']

char  int
a     1      0.825006
      2      0.326325
b     1      0.797434
      2      0.425433
dtype: float64

### Stacking and unstacking indices

In [105]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [113]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [114]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [115]:
pop.unstack(level=1).stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Index setting and resetting

Another way is to turn the index labels into columns, using reset_index. We can also give the data a name:

In [116]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [119]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


If we want to convert this into a MultiIndex series we can do the following:

In [120]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## Data Aggregations

We can pass a level/code parameter to the aggregation methods to choose which level to aggregate on.

In [123]:
health_data

Subject      Bob       Guido         Sue      
Type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.0  37.3  39.0  35.9  28.0  37.2
     2      31.0  37.2  37.0  38.7  43.0  37.1
2014 1      14.0  36.2  39.0  37.2  32.0  36.1
     2      26.0  37.4  22.0  35.7  34.0  36.6

In [129]:
data_mean = health_data.mean(level='year')
data_mean

Subject   Bob        Guido          Sue       
Type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     34.0  37.25  38.0  37.30  35.5  37.15
2014     20.0  36.80  30.5  36.45  33.0  36.35

Using the axis keyword we can take the mean among levels on the columns as well:

In [135]:
data_mean.mean(axis=1, level='Type')

Type,HR,Temp
year,,
2013,35.833333,37.233333
2014,27.833333,36.533333
